In [1]:
from datetime import datetime, timedelta, timezone
from sqlalchemy import create_engine
from common.utils import *
from database.utils import *
from pages.market_overview_light import *

from datetime import datetime

2023-12-15 15:45:34.850 
  command:

    streamlit run /Users/jazzdung/projects/streamlit-server/.venv/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]
2023-12-15 15:45:34.852 No runtime found, using MemoryCacheStorageManager
2023-12-15 15:45:34.857 No runtime found, using MemoryCacheStorageManager
2023-12-15 15:45:34.860 No runtime found, using MemoryCacheStorageManager
2023-12-15 15:45:34.864 No runtime found, using MemoryCacheStorageManager
2023-12-15 15:45:38.078 No runtime found, using MemoryCacheStorageManager
2023-12-15 15:45:38.335 No runtime found, using MemoryCacheStorageManager
2023-12-15 15:45:38.700 No runtime found, using MemoryCacheStorageManager
2023-12-15 15:45:39.825 No runtime found, using MemoryCacheStorageManager


In [4]:
latest_price_metric = get_latest_record(schema='public', table='price_metric_all_resolution', engine=dsai_engine)

latest_price_metric.query('name == "market"')[[
    'price_increased_sum',
    'price_decreased_sum',
    'price_increased_mean',
    'price_decreased_mean',
]].values.flatten().tolist()

[338.0, 445.0, 0.20954742715437075, 0.2758834469931804]

In [1]:
timestamp = datetime(2023, 10, 16)
sql = f"""
    select
        *
    from public.liquidity_all_resolution
    where indexed_timestamp_ > '{timestamp.strftime('%Y-%m-%d')}'::date - interval '5 days'
    and indexed_timestamp_::date <= '{timestamp.strftime('%Y-%m-%d')}'::date
    order by indexed_timestamp_;
    """
df = pd.read_sql_query(
    sql=sql,
    con=dsai_engine
)

df["indexed_timestamp_"] = pd.to_datetime(df["indexed_timestamp_"])
df = df[df['resolution'] == 'market'] 
df['minute'] = df['indexed_timestamp_'].apply(lambda x: x.minute)
df['time'] = df['indexed_timestamp_'].dt.time
df['time_str'] = df['indexed_timestamp_'].dt.strftime('%H:%M')

df = df[df['minute'].isin([0, 15, 30, 45])]
df = df.sort_values(by='indexed_timestamp_', ascending=False)

today = df[df['indexed_timestamp_'].dt.date == timestamp.date()]

five_day_ago = df[df['indexed_timestamp_'].dt.date < timestamp.date()]
five_day_ago = five_day_ago.groupby('time').agg(
    max_liquidity=('liquidity', 'max'), 
    min_liquidity=('liquidity', 'min')
)

five_day_ago.reset_index(inplace=True)
today.reset_index(inplace=True)

True

In [144]:
today

,index,indexed_timestamp_,liquidity,name,resolution,liquidity_1d_ago,liquidity_avg_7d_ago,liquidity_vs_1d_ago,liquidity_vs_avg_7d_ago,minute,time,time_str
0,10500,2023-10-16 08:45:00,28883.328454,Thị trường,market,None,None,None,None,45,08:45:00,08:45
1,10380,2023-10-16 08:00:00,28883.032202,Thị trường,market,None,None,None,None,0,08:00:00,08:00
2,10200,2023-10-16 07:45:00,27597.979855,Thị trường,market,None,None,None,None,45,07:45:00,07:45
3,10020,2023-10-16 07:30:00,27551.889950,Thị trường,market,None,None,None,None,30,07:30:00,07:30
4,9840,2023-10-16 07:15:00,24929.323811,Thị trường,market,None,None,None,None,15,07:15:00,07:15
5,9664,2023-10-16 07:00:00,23292.854002,Thị trường,market,None,None,None,None,0,07:00:00,07:00
6,9490,2023-10-16 06:45:00,22359.188434,Thị trường,market,None,None,None,None,45,06:45:00,06:45
7,9307,2023-10-16 06:30:00,21551.343798,Thị trường,market,None,None,None,None,30,06:30:00,06:30
8,9136,2023-10-16 06:15:00,20428.441790,Thị trường,market,None,None,None,None,15,06:15:00,06:15
9,8881,2023-10-16 04:30:00,19317.479098,Thị trường,market,None,None,None,None,30,04:30:00,04:30


In [145]:
five_day_ago

,time,max_liquidity,min_liquidity
0,00:45:00,0.000000,0.000000
1,02:15:00,625.549872,166.172950
2,02:30:00,1642.217539,1331.951198
3,02:45:00,2920.847783,2095.826372
4,03:00:00,3940.475163,2866.243171
5,03:15:00,4724.444113,3609.947263
6,03:30:00,5443.300686,4172.723171
7,03:45:00,6178.608481,4557.499541
8,04:00:00,6745.270097,4938.543123
9,04:15:00,7348.490533,5335.622806


In [137]:
timestamp = datetime(2023, 10, 16)
sql = f"""
    select
        *
    from public.liquidity_all_resolution
    where indexed_timestamp_::date = '{timestamp.strftime('%Y-%m-%d')}'::date
    order by indexed_timestamp_;
    """
df = pd.read_sql_query(
    sql=sql,
    con=dsai_engine
)

df["indexed_timestamp_"] = pd.to_datetime(df["indexed_timestamp_"])
df = df[df['resolution'] == 'sector'] 
df['minute'] = df['indexed_timestamp_'].apply(lambda x: x.minute)
df['time'] = df['indexed_timestamp_'].dt.time

df = df[df['minute'].isin([0, 15, 30, 45])]
df = df.sort_values(by='indexed_timestamp_', ascending=False)

top_8_liquidity = df[df['indexed_timestamp_'] == df['indexed_timestamp_'].max()].sort_values(by='liquidity', ascending=False).head(8)['name'].unique().tolist()
today = df[df['indexed_timestamp_'].dt.date == timestamp.date()]
today = today[today['name'].isin(top_8_liquidity)]

today['ratio'] = today['liquidity'] / today.groupby('time')['liquidity'].transform(pd.Series.sum)

In [138]:
today

,indexed_timestamp_,liquidity,name,resolution,liquidity_1d_ago,liquidity_avg_7d_ago,liquidity_vs_1d_ago,liquidity_vs_avg_7d_ago,minute,time,ratio
3375,2023-10-16 08:45:00,1124.901685,Bán lẻ đa kênh,sector,None,None,None,None,45,08:45:00,0.049702
3374,2023-10-16 08:45:00,1444.070214,Sản phẩm thực phẩm,sector,None,None,None,None,45,08:45:00,0.063803
3373,2023-10-16 08:45:00,1616.356698,Kim loại và khai thác,sector,None,None,None,None,45,08:45:00,0.071415
3372,2023-10-16 08:45:00,1929.668729,Hóa chất,sector,None,None,None,None,45,08:45:00,0.085259
3371,2023-10-16 08:45:00,2187.555355,Kỹ thuật xây dựng,sector,None,None,None,None,45,08:45:00,0.096653
...,...,...,...,...,...,...,...,...,...,...,...
132,2023-10-16 02:15:00,4.749860,Hóa chất,sector,None,None,None,None,15,02:15:00,0.076199
130,2023-10-16 02:15:00,11.766220,Kỹ thuật xây dựng,sector,None,None,None,None,15,02:15:00,0.188758
129,2023-10-16 02:15:00,16.860880,Môi giới chứng khoán và hàng hóa,sector,None,None,None,None,15,02:15:00,0.270489
128,2023-10-16 02:15:00,25.995850,Quản lý và Phát triển bất động sản,sector,None,None,None,None,15,02:15:00,0.417036


In [ ]:
df

In [ ]:
import streamlit as st
import time
import numpy as np
import pandas as pd 


conn = st.experimental_connection("postgresql", type="sql")
df = conn.query("select liquidity from liquidity_all_resolution where resolution = 'market' limit 100 ", ttl="10m")


# for row in df.itertuples():
#     st.write(f"{row.name} has a :{row.pet}:")


# chart_data = pd.DataFrame(np.random.randn(20, 3), columns=["a", "b", "c"])

st.line_chart(df)

In [198]:
# sector_info_sql = f"""
#     select
#         symbol_
#         , nganhcap3
#         , nganhcap2
#         , nganhcap1
#     from public.wifeed_bctc_thong_tin_co_ban_doanh_nghiep
#     where indexed_timestamp_ in (select max(indexed_timestamp_) from wifeed_bctc_thong_tin_co_ban_doanh_nghiep)
# """
# 
# sector_info = pd.read_sql_query(
#     sql=sector_info_sql,
#     con=dsai_engine
# )

timestamp = datetime(2023, 12, 12)
symbol_sql = f"""
    select *
    from public.symbol_all_metric
    where indexed_timestamp_ = (
        select max(indexed_timestamp_) 
        from public.symbol_all_metric 
        where indexed_timestamp_::date = '{timestamp.strftime('%Y-%m-%d')}')
"""

symbol = pd.read_sql_query(
    sql=symbol_sql,
    con=dsai_engine
)
symbol['net_value'] = symbol['buyForeignQttyVal'] - symbol['sellForeignQttyVal']

sector = symbol.groupby('sector')['net_buy'].apply(lambda x: (x > 0).sum()).reset_index(name='net_buy_count')
sector['net_sell_count'] = symbol.groupby('sector')['net_sell'].apply(lambda x: (x > 0).sum()).reset_index()['net_sell']
sector['net_value'] = symbol.groupby('sector')['net_value'].apply(lambda x: x.sum()).reset_index()['net_value']
sector['symbol_count'] = symbol.groupby('sector')['symbol'].apply(lambda x: x.count()).reset_index()['symbol']
sector['buy_minus_sell_count'] = sector['net_buy_count'] - sector['net_sell_count']
sector['net_buy_ratio'] = sector['net_buy_count'] / sector['symbol_count']
sector['net_sell_ratio'] = sector['net_sell_count'] / sector['symbol_count']



In [196]:
symbol

,symbol,sector,floor,buyForeignQtty,sellForeignQtty,buyForeignQttyVal,sellForeignQttyVal,liquidity,net_buy,net_sell,indexed_timestamp_,net_value
0,A32,"Hàng dệt, may và phụ kiện",UPCOM,0.0,0.0,0.0,0.000,0.036070,0.0,0.000,2023-12-12 08:05:00,0.000
1,AAA,Hóa chất,HOSE,70.0,14320.6,678.3,138688.914,29.663050,0.0,138010.614,2023-12-12 08:05:00,-138010.614
2,AAM,Sản phẩm thực phẩm,HOSE,0.0,0.0,0.0,0.000,0.054442,0.0,0.000,2023-12-12 08:05:00,0.000
3,AAS,Môi giới chứng khoán và hàng hóa,UPCOM,0.0,0.0,0.0,0.000,6.133890,0.0,0.000,2023-12-12 08:05:00,0.000
4,AAT,"Hàng dệt, may và phụ kiện",HOSE,0.0,0.0,0.0,0.000,1.571661,0.0,0.000,2023-12-12 08:05:00,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...
927,XMC,Kỹ thuật xây dựng,UPCOM,0.0,0.0,0.0,0.000,0.020960,0.0,0.000,2023-12-12 08:05:00,0.000
928,YBC,Vật liệu xây dựng,UPCOM,10.0,0.0,73.0,0.000,0.000730,73.0,0.000,2023-12-12 08:05:00,73.000
929,YBM,Kim loại và khai thác,HOSE,0.0,0.0,0.0,0.000,0.060210,0.0,0.000,2023-12-12 08:05:00,0.000
930,YEG,Phương tiện truyền thông,HOSE,1130.0,370.0,20733.5,6787.500,16.747690,13946.0,0.000,2023-12-12 08:05:00,13946.000


In [199]:
sector

,sector,net_buy_count,net_sell_count,net_value,symbol_count,buy_minus_sell_count,net_buy_ratio,net_sell_ratio
0,Bao bì,0,0,0.000000e+00,7,0,0.000000,0.000000
1,Bán lẻ chuyên dụng,2,0,2.371876e+05,12,2,0.166667,0.000000
2,Bán lẻ đa kênh,1,1,3.821882e+05,4,0,0.250000,0.250000
3,Bảo hiểm và hoạt động liên quan,6,3,-1.333685e+05,12,3,0.500000,0.250000
4,Cơ sở hạ tầng giao thông,3,7,-2.961883e+06,31,-4,0.096774,0.225806
5,Dược phẩm,2,4,-1.151120e+05,26,-2,0.076923,0.153846
6,"Dầu, khí đốt và nhiên liệu tiêu hao",2,0,2.144912e+06,5,2,0.400000,0.000000
7,Dịch vụ IT,0,2,-1.618640e+04,3,-2,0.000000,0.666667
8,Dịch vụ tiêu dùng đa dạng,0,0,0.000000e+00,1,0,0.000000,0.000000
9,Dịch vụ tài chính đa dạng,1,0,3.104400e+03,1,1,1.000000,0.000000


In [163]:
symbol[symbol['sector'] == 'Bảo hiểm và hoạt động liên quan']

,symbol,sector,floor,buyForeignQtty,sellForeignQtty,buyForeignQttyVal,sellForeignQttyVal,liquidity,net_buy,net_sell,indexed_timestamp_,net_value
4,ABI,Bảo hiểm và hoạt động liên quan,UPCOM,150.0,7.2,3900.0,187.2,0.49984,3712.8,0.0,2023-12-13 07:15:00,None
8,AIC,Bảo hiểm và hoạt động liên quan,UPCOM,0.0,0.0,0.0,0.0,0.13793,0.0,0.0,2023-12-13 07:15:00,None
32,BHI,Bảo hiểm và hoạt động liên quan,UPCOM,0.0,0.0,0.0,0.0,0.02646,0.0,0.0,2023-12-13 07:15:00,None
36,BLI,Bảo hiểm và hoạt động liên quan,UPCOM,40.0,0.0,408.0,0.0,0.12268,408.0,0.0,2023-12-13 07:15:00,None
347,PTI,Bảo hiểm và hoạt động liên quan,HNX,4670.0,4670.0,0.0,0.0,0.01498,0.0,0.0,2023-12-13 07:15:00,None
353,PVI,Bảo hiểm và hoạt động liên quan,HNX,50.0,0.0,2260.0,0.0,0.18045,2260.0,0.0,2023-12-13 07:15:00,None
529,VNR,Bảo hiểm và hoạt động liên quan,HNX,0.0,0.0,0.0,0.0,0.02773,0.0,0.0,2023-12-13 07:15:00,None


In [ ]:
import pandas as pd
sector_metric = load_sector_metric(start=datetime.now() - timedelta(days=15), end=datetime.now())
top_10_sector = sector_metric.groupby(by=['ten_nganh']).agg({'total_thanh_khoan': 'sum'}).sort_values(by='total_thanh_khoan', ascending=False).head(10)
sector_metric = sector_metric[ sector_metric['ten_nganh'].isin(top_10_sector.index)]
sector_metric_total_cap = sector_metric.groupby(by=['ngay']).agg({'total_thanh_khoan': 'sum'})
sector_metric = pd.merge(sector_metric, sector_metric_total_cap, on=['ngay','ngay'], suffixes=['', '_all'])
sector_metric['sector_liquidity_ratio'] = sector_metric['total_thanh_khoan'] / sector_metric['total_thanh_khoan_all']
sector_metric

In [ ]:
def color_decider(value, threshold=None, threshold_color=None):
    if threshold is None:
        threshold = [0]
    if threshold_color is None:
        threshold_color = ['red', 'green']
    
    color = threshold_color[0]
    for i in range(len(threshold)):
        if value >= threshold[i]:
            color = threshold_color[min(len(threshold_color) - 1, i+1)]
    return color

color_decider(value=5, threshold=[2,4], threshold_color=['green', 'red'])

In [ ]:
def load_market_price_metric_range(start, end):
    start_date_string = datetime.strftime(start_date, '%Y-%m-%d')
    end_date_string = datetime.strftime(end_date, '%Y-%m-%d')
    sql = f"""
            select
                indexed_timestamp_
                , avg(pha_day_1m) as pha_day_1m
                , avg(vuot_dinh_1m) as vuot_dinh_1m
            from dsai.dnse_thong_ke_cuoi_ngay_1d0h0m_1
            where 
                indexed_timestamp_::date >= '{start_date_string}'
                and indexed_timestamp_::date <= '{end_date_string}'
            group by indexed_timestamp_
            order by  indexed_timestamp_ desc
        """
    df = pd.read_sql_query(
        sql=sql,
        con=redshift_engine
    )
    return df


load_market_price_metric_range

In [ ]:
all_liquidity_same_period = get_last_record_of_day(schema='public', table='liquidity_all_resolution', engine=dsai_engine)
all_liquidity_same_period

In [ ]:
get_latest_record(schema='public', table='liquidity_all_resolution', engine=engine).query(
    '(resolution == "market" or resolution == "sector")'
).sort_values(by='liquidity', ascending=False)[['name']].values.flatten().tolist()[:4]

In [ ]:
market_and_top_n_liquidity = all_liquidity_same_period.query(
    '(resolution == "market" or resolution == "sector")'
).head(3 + 1)

market_and_top_n_liquidity

In [ ]:
latest_price_metric.query('name == "market"')[['price_increased_sum', 'price_decreased_sum']].values.flatten().tolist()

In [ ]:
from configs.config import DSAI_TIMESCALEDB_PRD
from sqlalchemy import create_engine
from database.utils import get_record_latest
engine = create_engine(DSAI_TIMESCALEDB_PRD)
record = get_record_latest(
    schema='public',
    table='liquidity_all_resolution',
    engine=engine
)

In [ ]:
record